In [2]:
import pandas as pd
import datetime as dt
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

In [3]:
path = "./DataLab2.xlsx"
df = pd.read_excel(path, index_col = 0)
estimates_df = df[500:].reset_index().set_index('Date').rename(
    columns={'ESEWMA': 'ESewma', 'ESPot': 'ESpot'}
)
estimates_df.head()

,PL,VaRBHS,VaREWMA,VaRn,VaRt,VaRPot,ESewma,ESn,ESt,ESpot
Date,,,,,,,,,,
2006-12-26,-150,1115.0,1147.464693,777.535296,986.872911,1391.726372,2544.116901,897.215494,3209.017239,1924.333523
2006-12-28,150,1115.0,1144.195603,778.321836,987.104816,1391.726372,2540.137912,898.017553,3189.914672,1924.333523
2006-12-29,-20,1115.0,1117.460475,778.334975,987.885905,1391.726372,2506.631070,898.020953,3203.255498,1924.333523
2006-12-31,30,1115.0,1085.901247,777.503184,989.976454,1391.726372,2468.406961,897.126264,3261.771712,1924.333523
2007-01-01,30,1115.0,1053.756229,777.645897,990.797723,1391.726372,2431.278879,897.263546,3270.873551,1924.333523


In [5]:
violations_df = pd.DataFrame(estimates_df["VaRBHS"] < -estimates_df["PL"], columns = ["vio_bhs"])
violations_df["vio_ewma"] = estimates_df["VaREWMA"] < -estimates_df["PL"]
violations_df["vio_n"] = estimates_df["VaRn"] < -estimates_df["PL"]
violations_df["vio_t"] = estimates_df["VaRt"] < -estimates_df["PL"]
violations_df["vio_pot"] = estimates_df["VaRPot"] < -estimates_df["PL"]

violations_df[violations_df.vio_ewma].sample(5)

,vio_bhs,vio_ewma,vio_n,vio_t,vio_pot
Date,,,,,
2007-07-07,False,True,True,True,False
2008-07-10,False,True,True,True,False
2008-09-26,False,True,True,False,False
2008-10-10,True,True,True,True,True
2007-07-10,True,True,True,True,True


In [24]:
year2007 = (violations_df.index >= pd.Timestamp('2007-01-01')) & (violations_df.index <= pd.Timestamp('2007-12-31'))
year2008 = (violations_df.index >= pd.Timestamp('2008-01-01')) & (violations_df.index <= pd.Timestamp('2008-12-31'))

n_violations = {('bhs',2007): violations_df[year2007].vio_bhs.sum(),
               ('ewma',2007): violations_df[year2007].vio_ewma.sum(),
                ('n',2007): violations_df[year2007].vio_n.sum(),
                ('t',2007): violations_df[year2007].vio_t.sum(),
                ('pot',2007): violations_df[year2007].vio_pot.sum(),
                
                ('bhs',2008): violations_df[year2008].vio_bhs.sum(),
               ('ewma',2008): violations_df[year2008].vio_ewma.sum(),
                ('n',2008): violations_df[year2008].vio_n.sum(),
                ('t',2008): violations_df[year2008].vio_t.sum(),
                ('pot',2008): violations_df[year2008].vio_pot.sum(),
               }
n_violations

{('bhs', 2007): 15,
 ('ewma', 2007): 5,
 ('n', 2007): 36,
 ('t', 2007): 27,
 ('pot', 2007): 5,
 ('bhs', 2008): 7,
 ('ewma', 2008): 6,
 ('n', 2008): 12,
 ('t', 2008): 9,
 ('pot', 2008): 3}

In [25]:
print(f"Expected amount of violations: {0.01*252}") #252 samples per year

Expected amount of violations: 2.52


In [20]:
p_values_kupiec = {k:1-stats.binom.cdf(v-1,252,0.01) for k, v in n_violations.items()}

print("Method:\tYear:\tKupiec probability:")
for (name,year) in p_values_kupiec: 
    print(f'{name}\t{year}\t{p_values_kupiec[(name,year)]:.4f}')

Method:	Year:	Kupiec probability:
bhs	2007	0.0000
ewma	2007	0.1105
n	2007	0.0000
t	2007	0.0000
pot	2007	0.1105
bhs	2008	0.0143
ewma	2008	0.0425
n	2008	0.0000
t	2008	0.0011
pot	2008	0.4620


In [1]:
def map_traffic_light(nbr_violations):
    if(nbr_violations < 5):
        color = "Green"
    elif (nbr_violations < 10):
        color = "Yellow"
    else:
        color = "Red"
    return color

In [9]:
print("Method:\tYear:\tTraffic light:")
for (name,year) in p_values_kupiec: 
    color = map_traffic_light(n_violations[(name,year)])
    print(f'{name}\t{year}\t{color}')

Method:	Year:	Traffic light:
bhs	2007	Red
ewma	2007	Yellow
n	2007	Red
t	2007	Red
pot	2007	Yellow
bhs	2008	Yellow
ewma	2008	Yellow
n	2008	Red
t	2008	Yellow
pot	2008	Green


In [11]:
# Christoffersen
# count all n00, n11, n01, n10, n1, n0
years = {2007: year2007, 2008: year2008}
method_names = ['bhs', 'ewma', 'n', 't', 'pot']
keys = [(name,year) for year in years for name in method_names]
christoff = dict(zip(keys,len(keys)*[None]))

n11 = ((violations_df['vio_bhs'] == True) & (violations_df['vio_bhs'].shift(-1) == True)).sum()

christoff['bhs',2007] = {'n11' : 1}

for year in years:
    for name in method_names:
        current_year = years[year]
        column_name = 'vio_' + name
        
        n11 = ((violations_df[current_year][column_name] == True ) & (violations_df[current_year][column_name].shift(-1) == True )).sum()
        n00 = ((violations_df[current_year][column_name] == False) & (violations_df[current_year][column_name].shift(-1) == False)).sum()
        n01 = ((violations_df[current_year][column_name] == False) & (violations_df[current_year][column_name].shift(-1) == True )).sum()
        n10 = ((violations_df[current_year][column_name] == True ) & (violations_df[current_year][column_name].shift(-1) == False)).sum()
        
        n0 = (violations_df[current_year][column_name] == False).sum()
        n1 = (violations_df[current_year][column_name] == True).sum()
        
        
        christoff[name,year] = {'n00' : n00, 
                                'n01' : n01, 
                                'n10' : n10,
                                'n11' : n11,
                                'n0' : n0,
                                'n1' : n1,
                               }

In [13]:
for (name,year) in christoff:
    
    n00 = christoff[(name,year)]['n00'] 
    n01 = christoff[(name,year)]['n01']
    n10 = christoff[(name,year)]['n10']
    n11 = christoff[(name,year)]['n11']
        
    n0 = christoff[(name,year)]['n0']
    n1 = christoff[(name,year)]['n1']
    
    pi00 = n00/(n00+n01) 
    pi01 = n01/(n00+n01)
    pi10 = n10/(n10+n11)
    pi11 = n11/(n10+n11)
        
    pi0 = n0/(n1+n0)
    pi1 = n1/(n1+n0) 
    
    if((pi0+pi1 != 1) or (pi10+pi11 != 1)):
        print(f"help! pls check {name} {year}")
    
    lnNull = np.log(pi0**n0*pi1**n1) 
    lnAlt = np.log(pi00**n00*pi01**n01*pi10**n10*pi11**n11) 
    LR_ind = -2*(lnNull-lnAlt)
    pVal = 1-stats.chi2.cdf(LR_ind,1)
    print(f"LR_ind for model: {name}, year: {year}")
    print(f"Test statistic \t P-value")
    print(f"{-2*(lnNull-lnAlt):.4f} \t\t {pVal:.4f}\n")   

LR_ind for model: bhs, year: 2007
Test statistic 	 P-value
1.3087 		 0.2526

LR_ind for model: ewma, year: 2007
Test statistic 	 P-value
9.9663 		 0.0016

LR_ind for model: n, year: 2007
Test statistic 	 P-value
5.5850 		 0.0181

LR_ind for model: t, year: 2007
Test statistic 	 P-value
5.9135 		 0.0150

LR_ind for model: pot, year: 2007
Test statistic 	 P-value
0.2434 		 0.6217

LR_ind for model: bhs, year: 2008
Test statistic 	 P-value
6.8231 		 0.0090

LR_ind for model: ewma, year: 2008
Test statistic 	 P-value
8.2158 		 0.0042

LR_ind for model: n, year: 2008
Test statistic 	 P-value
6.3087 		 0.0120

LR_ind for model: t, year: 2008
Test statistic 	 P-value
4.7217 		 0.0298

LR_ind for model: pot, year: 2008
Test statistic 	 P-value
5.4650 		 0.0194



In [18]:
es_df = estimates_df[['ESewma', 'ESn', 'ESt', 'ESpot']]

In [16]:
# Acerbi-Szekely test
print("Model:\tYear:\tValue of Z:")
for year in years:
    for name in method_names:
        if(name is 'bhs'):
            continue
            
        current_year = years[year]
        violation_name = 'vio_'+name
        
        ES_name = 'ES'+name
        
        alpha = 0.99
        M = len(violations_df[current_year])
        I = violations_df[current_year][violation_name]
        L = -estimates_df[current_year].PL
        ES = estimates_df[current_year][ES_name]
        
        Z = -sum(I*L/ES)/(M*(1-alpha))+1
        
        print(f"{name}\t{year}\t{Z:.4f}")


Model:	Year:	Value of Z:
ewma	2007	-0.4661
n	2007	-18.4620
t	2007	-1.3059
pot	2007	-1.1670
ewma	2008	-1.0487
n	2008	-6.5774
t	2008	0.1105
pot	2008	-0.5021


# Questions

## a) 
According to our tests, not really any model performs exceptionally well. 

From the Kupiec test, we see that we only "*accept*" (not reject) the VaR method POT for both 2007 and 2008 on a 5% level. If we are kind we allow EWMA since 2008 it almost survives on a 5% level.

From the traffic light test, no model is green both years. If we take yellow as an acceptable score, we see that EWMA and POT again performs decently.

From Christoffersen we see that no method produces a P-value above 0.05 both years. 

From Acerbi-Szekely we see that EWMA, t, and POT all produce decent values, that is being fairly close to 0.

## b)
For VAR I would pick POT, as it produced the overall best results and was the only to survive the Basel traffic light on a green level at any year. The color on traffic light is very important when it comes to return on equity, since a lower capital requirement lowers the amount of equity needed.

For ES I would pick EWMA since it produces an average absolute Z close to zero in a consistent way. t produces the lowest value but is less consistent, meaning that we cannot trust the value to remain low the coming years. EWMA also responds better to volatility changes.




In [ ]:
#Question A

#KUPIEC

#null hypothesis: the model's prediction is no better than random chance

#2007

#VaRBHS p-value 0,0000 Reject
#VarEWMA p-value 0,1132 Not reject at 1% level
#VaRt p-value 0.0000 Reject
#VaRn p-value 0,0000 Reject
#VaRPot p-value 0,1132 Not reject at 1% level

#2008

#VaRBHS p-value 0,0148 Not reject at 1% level
#VarEWMA p-value 0,0439 Not reject at 1% level
#VaRt p-value 0,0012 Reject
#VaRn p-value 0,0000 Reject
#VaRPot p-value 0,4671 Not reject at 1% level

#VaRt and VaRn are the only methods that are better than random chance for both the sample years.

#CHRISTOFFERSON

#null hypothesis: estimations of VaR is idd

#VarBHS 2007 LRD_IND is 1,3087 and the p-value is 0,2525. Not rejected. at 1%-level.
#EWMA 2007 LRD_IND is 9,9663 and the p-value is 0,0016. Reject at 1%-level.
#VaRn 2007 LRD_IND is 5,5850 and the p-value is 0,0181. Not rejected at 1%-level.
#VaRt 2007 LRD_IND is 5,9135 and the p-value is 0,0150. Not rejected at 1%-level.
#VaRPot 2007, Check1 = 1,000 Check2 = 0,714 LRD_IND is 5,0628 and the p-value is 0,0244. Not rejected at 1%-level.

#VarBHS 2008 LRD_IND is 6,8231 and the p-value is 0,0090. Null is rejected at 1%-level.
#EWMA 2008 LRD_IND is 8,2158 and the p-value is 0,0042. Rejected at 1%-level.
#VaRn 2008 LRD_IND is 6,3087 and the p-value is 0,0120. Not rejected at 1%-level.
#VaRt 2008 LRD_IND is 4,7217 and the p-value is 0,0298. Not rejected at 1%-level.
#VaRPot 2008, Check1 = 1,000 Check2 = 0,375 LRD_IND is 11,6209 and the p-value is 0,0007. Null is rejected at 1%-level

#Acerbi-Szekely 

#The null for the AS test is that the expected value of the Z2 score = 0 implying that the model correctly predicts ES for all periods t
#The counter is then that model, at least once, under estimates ES. 

#Question B

#Based on the tests VaRn and VaRt are the most accurate models for estimation.

#VaRn KUPIC 2007: 36 exceptions 2008: 12 exceptions
#VaRt KUPIC 2007: 27 exceptions 2008: 9 exceptions
#VaRt has a lower amount of violations for both 2007 and 2008 and therefore 
#overestimates VaR in comparison to VaRn, thus VaRt is a more suitable method for value at risk.

#Our resuluts indicate that none of the ES-models correctly specifies ES for all time-periods. The smallest deviation is an 
#overestimation from ESt of 0.11 in 2008.
#ESt performes relatively well for 2007 as well with an underestimation of -1.31

#Overestimating ES is from a risk perspective prefered over an underestimation since it implies that the realized loss is smaller than predicted.
#However from a profits perspective this would mean that a bank would have to carry more reserve capital in order to accommodate the ES.

#The model with the smallest spread is EWMA which underestimates ES for both 2007 (-0.466) and 2008 (-1.049)

#With these statements as motivation we believe that ESt is the best model for ES.